In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import os
import shutil

In [2]:
#list allk files in the input directory

input_dir = '/kaggle/input/cassava-leaf-disease-classification'
os.listdir(input_dir)

['train_tfrecords',
 'sample_submission.csv',
 'test_tfrecords',
 'label_num_to_disease_map.json',
 'train_images',
 'train.csv',
 'test_images']